### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [1]:
import pandas as pd
from SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Chargement des données**

In [2]:
data = updateInfoProfs()

**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [3]:
expertises = data[['idsadvr', 'affiliations', 'etablissementsAffilies', 'expertise']]

toNormalize = ['affiliations', 'etablissementsAffilies', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays',
                'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble',
        'affiliations.fonction.codeSad', 'affiliations.fonction.nom', 'affiliations.local', 
        'affiliations.exclusion', 'affiliations.exclusionTel','affiliations.uniteAdministrative.codeSad', 
        'affiliations.uniteAdministrative.nom', 'affiliations.telephone.numero', 'affiliations.telephone.poste']

expertises = expertises.drop(columns=drop)

*Établissements affiliés*

In [4]:
# Etablissements affiliés
etablissementsAffilies = expertises.dropna(subset='etablissementsAffilies.nom')
etablissementsAffilies = etablissementsAffilies.drop_duplicates(subset=(['idsadvr', 'etablissementsAffilies.nom']))
etablissementsAffilies = pd.DataFrame(plotVariable(etablissementsAffilies, 'etablissementsAffilies.nom')).sort_values(by='count')

etablissementsAffilies

,labels,count
26,Institut Lady Davis de de recherche médicale (...,1
24,CIUSS de la Mauricie-et-du-Centre-du-Québec – ...,1
23,Centre de recherche sur la dynamique du systèm...,1
25,CIUSSS du Centre-Sud-de-l’Île-de-Montréal – In...,1
22,CIUSS du Centre-Ouest-de-l’Île-de-Montréal – C...,2
21,Centre interuniversitaire de recherche sur la ...,3
20,CIUSSS du Centre-Sud-de-l’Île-de-Montréal – Ce...,3
19,CIUSS du Centre-Ouest-de-l’Île-de-Montréal – C...,3
18,CIUSSS du Centre-Sud-de-l’Île-de-Montréal – CS...,4
17,CIUSSS du Centre-Sud-de-l’Île-de-Montréal – CS...,4


In [5]:
px.bar(
    y = etablissementsAffilies['labels'],
    x = etablissementsAffilies['count'],
    orientation = 'h'
)

**Facultés, départements et établissements affiliés**

*Nombre de professeurs par facultés*

In [6]:
facultes = pd.DataFrame(plotVariable(expertises, 'affiliations.faculte.nom'))[:-2].sort_values(by='count')
facultes

,labels,count
11,École d'optométrie,31
10,Faculté de médecine dentaire,45
8,Faculté de l'aménagement,65
9,Faculté de musique,65
7,Faculté des sciences infirmières,67
6,Faculté de pharmacie,73
5,Faculté de droit,74
4,Faculté des sciences de l'éducation,79
3,Faculté de médecine vétérinaire,113
2,École de santé publique,174


In [8]:
px.bar(
    facultes, 
    x= facultes['count'], 
    y= facultes['labels'], 
    title='Nombre de professeur-e-s par facultés',
    width=1000, 
)

*Secteurs de recherche*

In [9]:
secteursRecherche = expertises[expertises['expertise.secteursRecherche.codeLangue'] == 'fre']
secteursRecherche = pd.DataFrame(plotVariable(secteursRecherche, 'expertise.secteursRecherche.nom')).sort_values(by='count')

secteursRecherche

,labels,count
24,Sciences de l’administration et de la gestion,17
22,Théologie et sciences des religions,27
23,Sciences de l’activité physique et réadaptation,27
21,Sciences humaines,38
20,Génie,43
19,Sciences sociales,46
18,Sciences pures,53
17,Communication,60
16,Sciences sociales : intervention,60
15,Sciences infirmières,63


In [10]:
px.bar(
    x = secteursRecherche['count'],
    y = secteursRecherche['labels'],
    orientation = 'h',
    height = 1000
)

**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / facultés / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

*Disciplines*

In [ ]:
disciplines = expertises[expertises['expertise.disciplines.codeLangue'] == 'fre']
disciplines = groupOtherValues(pd.DataFrame(plotVariable(disciplines, 'expertise.disciplines.nom')), 100)

disciplines

In [ ]:
neuro = expertises[expertises['expertise.disciplines.codeLangue'] == 'fre']
neuro = expertises[expertises['expertise.secteursRecherche.codeLangue'] == 'fre']

neuro = neuro[neuro['expertise.disciplines.nom'] == 'Neurosciences']
neuro
# motsClesNeuro = plotVariable(neuro, 'expertise.motsCles')
# pd.DataFrame(motsClesNeuro)